In [1]:
import disnake
from disnake.ext import commands
from disnake.ext.commands import has_guild_permissions
from disnake import Option, OptionType, Permissions
import json
import sys
import asyncio
import datetime
import os
from typing import Optional, Tuple


if sys.platform.startswith('win'):
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

intents = disnake.Intents.default()
intents.typing = False
intents.presences = False

TOKEN = os.environ.get("BOT_TOKEN")
GUILDS = [1043236594494230608]  # Remplacez par les ID de guildes souhaitées
CHANNEL_ID = 1093526619395723326

bot = commands.InteractionBot(intents=intents, test_guilds=GUILDS)

@bot.event
async def on_ready():
    print(f'Connecté en tant que {bot.user.name} - {bot.user.id}')

personnages_predefinis = []
personnages = {}
utilisateurs = {}
emprunts = {}

def save_data():
    with open("personnages.json", "w", encoding="utf-8") as f:
        json.dump({"predefinis": personnages_predefinis, "personnages": personnages, "utilisateurs": utilisateurs, "emprunts": emprunts}, f)

def ajouter_personnage_predefini(nom, competence):
    personnage = {"nom": nom, "competence": competence}
    personnages_predefinis.append(personnage)
    save_data()
    return f"Le personnage prédéfini '{nom}' a été ajouté avec succès !"

def est_personnage_disponible(nom: str) -> Tuple[bool, Optional[str]]:
    current_week = datetime.datetime.now().isocalendar()[1]
    for user_id, persos in personnages.items():
        for perso in persos:
            if perso["nom"].lower() == nom.lower() and perso["semaine"] == current_week:
                if perso["disponible"]:
                    return True, None
                else:
                    return False, perso["emprunteur_id"]
    return True, None

def est_personnage_de_utilisateur(personnage, user_id):
    if user_id not in utilisateurs:
        return False
    if "personnages" not in utilisateurs[user_id]:
        return False
    for perso in utilisateurs[user_id]["personnages"]:
        if perso["nom"].lower() == personnage.lower():
            return True
    return False

def ajouter_utilisateur_si_necessaire(user_id):
    if user_id not in utilisateurs:
        utilisateurs[user_id] = {
            "personnages": [],
            "emprunts": {}  # Changez cette ligne pour utiliser un dictionnaire
        }
        save_data()


def load_data():
    global personnages_predefinis
    global personnages
    global utilisateurs
    global emprunts
    try:
        with open("personnages.json", "r", encoding='utf-8') as f:
            data = json.load(f)
        if "predefinis" in data:
            personnages_predefinis = [personnage for personnage in data["predefinis"]]
        if "personnages" in data:
            personnages = data["personnages"]
        if "utilisateurs" in data:
            utilisateurs = data["utilisateurs"]
        if "emprunts" in data:
            emprunts = data["emprunts"]
    except json.JSONDecodeError:
        personnages_predefinis = []
        personnages = {}
        utilisateurs = {}
        emprunts = {}

def personnage_to_json(personnage):
    return {
        "nom": personnage["nom"],
        "competence": personnage["competence"]
    }

def is_admin_or_mod():
    async def predicate(inter: disnake.ApplicationCommandInteraction):
        return inter.author.guild_permissions.administrator or inter.author.guild_permissions.manage_roles
    return disnake.check(predicate)

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

@bot.slash_command(name="ajout_bdd", description="Ajouter un personnage à la base de données", guild_ids=GUILDS)
async def ajouter_personnage_predefini(
    inter: disnake.ApplicationCommandInteraction,
    nom: str = Option(name="nom", description="Le nom du personnage", type=OptionType.string, required=True),
    competence: str = Option(name="competence", description="La compétence du personnage", type=OptionType.string, required=True),
):
    # Vérifier si l'utilisateur a les permissions d'administrateur ou de modérateur
    if not (inter.author.guild_permissions.administrator or inter.author.guild_permissions.manage_roles):
        await inter.response.send_message("Vous n'avez pas les permissions nécessaires pour ajouter un personnage à la liste pré-définie.", ephemeral=True)
        return

    # Vérifier si les champs "nom" et "compétence" sont vides
    if nom == "" or competence == "":
        await inter.response.send_message("Les champs 'nom' et 'compétence' ne doivent pas être vides.", ephemeral=True)
        return

    # Vérifier si le nom du personnage n'existe pas déjà dans la base de données
    noms_autorises = [pers["nom"] for pers in personnages_predefinis]
    if nom.lower() in [nom_perso.lower() for nom_perso in noms_autorises]:
        await inter.response.send_message(f"Le personnage '{nom}' existe déjà dans la base de données.", ephemeral=True)
        return

    # Ajouter le personnage à la liste pré-définie et enregistrer les données
    personnages_predefinis.append({"nom": nom, "competence": competence})
    save_data()
    await inter.response.send_message(f"Le personnage '{nom}' a été ajouté à la liste pré-définie.", ephemeral=True)


# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

# Commande pour ajouter des personnages pour pret
@bot.slash_command(name="proposer", description="Ajouter un personnage à prêter")
async def ajouter_perso(
    inter: disnake.ApplicationCommandInteraction,
    nom: str = Option(name="nom", description="Le nom du personnage", type=OptionType.string, required=True)
):
    nom = nom.lower()

    noms_autorises = [pers["nom"].lower() for pers in personnages_predefinis]
    if nom not in noms_autorises:
        await inter.response.send_message(f"Le personnage '{nom.capitalize()}' n'est pas dans la BDD.", ephemeral=True)
        return

    disponible, user_id = est_personnage_disponible(nom)
    if not disponible:
        user = await bot.fetch_user(user_id)
        await inter.response.send_message(f"Le personnage '{nom.capitalize()}' n'est pas disponible. Veuillez contacter {user.mention} pour emprunter ce personnage.", ephemeral=True)
        return

    current_week = datetime.datetime.now().isocalendar()[1]
    user_id = str(inter.author.id)
    for perso in personnages.get(user_id, []):
        if perso["nom"].lower() == nom and perso["semaine"] == current_week:
            await inter.response.send_message(f"Le personnage '{nom.capitalize()}' est déjà dans votre liste pour cette semaine.", ephemeral=True)
            return

    for emprunt in utilisateurs.get(user_id, {}).get("emprunts", []):
        if emprunt["nom"].lower() == nom and emprunt["semaine"] == current_week - 1:
            await inter.response.send_message(f"Le personnage '{nom.capitalize()}' a déjà été prêté par vous la semaine dernière. Veuillez le proposer à un autre membre de la guilde.", ephemeral=True)
            return

    personnages[user_id] = [perso for perso in personnages.get(user_id, []) if perso["nom"].lower() != nom or perso["semaine"] == current_week]

    personnage = [pers for pers in personnages_predefinis if pers["nom"].lower() == nom][0]
    personnages[user_id].append({"nom": personnage["nom"].capitalize(), "competence": personnage["competence"], "semaine": current_week, "disponible": True, "emprunteur_id": None})
    save_data()
    await inter.response.send_message(f"Le personnage '{nom.capitalize()}' a été ajouté à votre liste pour la semaine {current_week}.", ephemeral=True)


# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

# Commande pour afficher la liste des personnages d'un utilisateur
@bot.slash_command(name="consulter_persos", description="Afficher la liste des personnages disponibles pour un pret")
async def liste_persos(inter: disnake.ApplicationCommandInteraction):
    current_week = datetime.datetime.now().isocalendar()[1]
    user_id = str(inter.author.id)

    # Vérifier si l'utilisateur a déjà emprunté un personnage la semaine précédente
    if user_id not in utilisateurs:
        utilisateurs[user_id] = {"emprunts": {}}
    emprunts_semaine_precedente = utilisateurs[user_id]["emprunts"].get(current_week - 1, [])

    perso_dispo_list = []
    for user_id, persos in personnages.items():
        for perso in persos:
            # Vérifier si le personnage est disponible pour emprunt
            if perso["nom"].lower() not in [p["nom"].lower() for p in perso_dispo_list]:
                dispo, emprunteur = est_personnage_disponible(perso["nom"].lower())
                if dispo and perso["disponible"] and (emprunteur is None or emprunteur.id != inter.author.id) and perso["nom"] not in emprunts_semaine_precedente:
                    perso_dispo_list.append({
                        "nom": perso["nom"].capitalize(),
                        "competence": perso["competence"],
                        "proprietaire": f"<@{user_id}>"
                    })

    if not perso_dispo_list:
        await inter.response.send_message("Aucun personnage n'est actuellement disponible.", ephemeral=True)
        return

    perso_dispo_list_str = "\n".join([f"- {perso['nom']} ({perso['competence']}) (Disponible chez {perso['proprietaire']})" for perso in perso_dispo_list])
    await inter.response.send_message(f"Liste des personnages disponibles :\n{perso_dispo_list_str}", ephemeral=True)

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

# Commande pour reserver en fonction de la liste des personnages d'un utilisateur
@bot.slash_command(name="need", description="Réserver un ou plusieurs personnages pour un ou plusieurs joueurs")
async def reserver_persos(
    inter: disnake.ApplicationCommandInteraction,
    personnage1: str = Option(name="personnage1", description="Le nom du premier personnage", type=OptionType.string, required=True),
    joueur1: disnake.Member = Option(name="joueur1", description="Le premier joueur", type=OptionType.user, required=True),
):
    # Vérifier si le personnage est déjà réservé
    for user_id, data in utilisateurs.items():
        for semaine, perso in data["emprunts"].items():
            if perso.lower() == personnage1.lower():
                emprunteur = await bot.fetch_user(int(user_id))
                await inter.response.send_message(f"Le personnage '{personnage1.capitalize()}' est déjà réservé par {emprunteur.mention}.", ephemeral=True)
                return

    # Vérifier si l'utilisateur essaie de réserver son propre personnage
    if est_personnage_de_utilisateur(personnage1, str(joueur1.id)):
        await inter.response.send_message(f"Vous ne pouvez pas réserver votre propre personnage.", ephemeral=True)
        return

    # Réserver le personnage
    dispo, emprunteur_id = est_personnage_disponible(personnage1)
    if dispo:
        # Ajouter le personnage aux emprunts de l'utilisateur
        user_id = str(joueur1.id)
        ajouter_utilisateur_si_necessaire(user_id)
        current_week = datetime.datetime.now().isocalendar()[1]
        utilisateurs[user_id]["emprunts"][str(current_week)] = personnage1
        save_data()

        # Obtenir le propriétaire du personnage
        proprietaire = None
        for uid, persos in personnages.items():
            for perso in persos:
                if perso["nom"].lower() == personnage1.lower() and perso["semaine"] == current_week:
                    proprietaire = await bot.fetch_user(int(uid))
                    break
            if proprietaire:
                break

        await inter.response.send_message(f"Le personnage '{personnage1.capitalize()}' a été réservé avec succès pour {joueur1.mention}.", ephemeral=False)
        await bot.get_channel(CHANNEL_ID).send(f"{joueur1.mention} a réservé le personnage '{personnage1.capitalize()}' prêté par {proprietaire.mention}.")
    else:
        emprunteur = await bot.fetch_user(int(emprunteur_id))
        await inter.response.send_message(f"Le personnage '{personnage1.capitalize()}' n'est pas disponible. Veuillez contacter {emprunteur.mention} pour emprunter ce personnage.", ephemeral=True)

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

# Commande pour supprimer un personnage de la liste d'un utilisateur
@bot.slash_command(name="supprimer_perso", description="Supprimer un personnage de votre liste")
async def supprimer_perso(
    inter: disnake.ApplicationCommandInteraction,
    nom: str = Option(name="nom", description="Le nom du personnage", type=OptionType.string, required=True)
):
    user_id = str(inter.author.id)
    if user_id not in personnages:
        await inter.response.send_message("Vous n'avez pas encore de personnages dans votre liste.", ephemeral=True)
        return

    for i, perso in enumerate(personnages[user_id]):
        if perso["nom"].lower() == nom.lower() and perso in personnages[user_id]:
            personnages[user_id] = [p for p in personnages[user_id] if p["nom"].lower() != nom.lower()]
            save_data()
            await inter.response.send_message(f"Le personnage '{nom.capitalize()}' a été supprimé de votre liste.", ephemeral=True)
            return

    await inter.response.send_message(f"Le personnage '{nom.capitalize()}' n'est pas dans votre liste.", ephemeral=True)

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

# Démarrage du bot
if __name__ == "__main__":
    load_data()  # Charger les données des personnages
    bot.run(TOKEN)

IndentationError: expected an indented block (2453818718.py, line 15)